In [1]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."

import json
import os
from ibm_watson import CompareComplyV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

import pandas as pd 
import text_extensions_for_pandas as tp
from IPython.core.display import HTML


## Initialize data and pre-process through watson Compare & Comply API

Run some initialization code, and process tables either through watson api, or load a cached version of such tables, for ease of use.

In [2]:
def init_watson_table_api():
    # Retrieve the APIKEY for authentication
    apikey = os.environ.get("IBM_API_KEY_TABLES")
    if apikey is None:
        raise ValueError("Expected apikey in the environment variable 'IBM_API_KEY'")

    # Get the service URL for your IBM Cloud instance
    ibm_cloud_service_url = os.environ.get("IBM_SERVICE_URL_TABLES")
    if ibm_cloud_service_url is None:
        raise ValueError("Expected IBM cloud service URL in the environment variable 'IBM_SERVICE_URL'")

        #initialize the authenticator
    authenticator = IAMAuthenticator(apikey)
    compare_comply = CompareComplyV1(
        version = '2020-1-1',
        authenticator=authenticator
    )

    compare_comply.set_service_url(ibm_cloud_service_url)
    return compare_comply

In [3]:
base_example_path = "../resources/tables/"
file_names = ["20-populous-countries", "cali-temp-chart", "california-population-chart", "double_header_table", "who_covid_report_table"]

archive_name = "archive"
#set to true to query watson
re_query_watson = False



In [4]:
#load in files either by querying cloud or from archive. 
if re_query_watson:
    print("This shouldn't happen")
    compare_comply = init_watson_table_api()
    for file_n in file_names:
        with open(f"{base_example_path}{file_n}.pdf", 'rb') as base_file:
            try:
                result = compare_comply.extract_tables(base_file).get_result()
                responses_dict[file_n] = result
            except ApiException as ex:
                print ("Method failed with status code " + str(ex.code) + ": " + ex.message)
     #archive file
    with open(f"{base_example_path}{archive_name}.json", 'w') as archive_file:
        json.dump(responses_dict, archive_file)
else:
    with open(f"{base_example_path}{archive_name}.json", 'r') as archive_file:
        responses_dict = json.load(archive_file)

## Now demo table extraction on a number of different tables.
To show how this tool can work in a variety of circumstances, here we demo using it on a number of different tables. 

First we display the reconstructed table, then we display the html that is created as an intermediate step of the table extraction, for comparison 

In [9]:
for name in file_names:
    print(f"\n\n Displaying table {name}:")
    print("\n Displaying Dataframe: \n")
    parsed = tp.watson_tables_parse_response(responses_dict[name])
    display(tp.make_exploded_df(parsed)[0])
    display(tp.make_table(parsed, concat_with=''))
    print("\n\n\n HTML: \n\n")
    display(HTML(tp.get_raw_html(responses_dict[name],parsed)))



 Displaying table 20-populous-countries:

 Displaying Dataframe: 



,text,attributes.type,row_index,column_header_texts_0
0,1,[Number],1,Rank
1,China [b],[Location],1,Country (or\ndependent\nterritory)
2,"1,403,627,360",[Number],1,Population
3,18.0%,[Percentage],1,% of worldpopulation
4,21 Jul 2020,[DateTime],1,Date
...,...,...,...,...
121,World,[],21,Country (or\ndependent\nterritory)
122,"7,800,767,000",[Number],21,Population
123,100%,[Percentage],21,% of worldpopulation
124,21 Jul 2020,[DateTime],21,Date


ERROR READING VALUE:""	 Filling with <NA>


,Rank,Country (or\ndependent\nterritory),Population,% of worldpopulation,Date,Source
1,1,China [b],1.403627e+09,18.00,21 Jul 2020,National populationclock [3]
2,2,India [c],1.364965e+09,17.50,21 Jul 2020,National populationclock [4]
3,3,United States [d],3.299913e+08,4.23,21 Jul 2020,National population\nclock [5]
4,4,Indonesia,2.696034e+08,3.46,1 Jul 2020,National annualprojection [6]
5,5,Pakistan [e],2.208923e+08,2.83,1 Jul 2020,UN Projection [2]
6,6,Brazil,2.118221e+08,2.72,21 Jul 2020,National populationclock [7]
7,7,Nigeria,2.061396e+08,2.64,1 Jul 2020,UN Projection [2]
8,8,Bangladesh,1.689908e+08,2.17,21 Jul 2020,National populationclock [8]
9,9,Russia [f],1.467486e+08,1.88,1 Jan 2020,National estimate [9]
10,10,Mexico,1.277923e+08,1.64,1 Jul 2020,National annualprojection [10]





 HTML: 




...



 Displaying table who_covid_report_table:

 Displaying Dataframe: 



,text,attributes.type,row_index,column_header_texts_0,column_header_texts_1
0,Africa,[Location],2,Reporting Country/ Territory/Area,
1,,[],2,Total confirmed,cases
2,,[],2,Total confirmed,new cases
3,,[],2,Total deaths,
4,,[],2,Total new deaths,
...,...,...,...,...,...
169,60,[Number],23,Total confirmed,new cases
170,26,[Number],23,Total deaths,
171,1,[Number],23,Total new deaths,
172,Community transmission,[],23,Transmission,classification i


ERROR READING VALUE:""	 Filling with <NA>
ERROR READING VALUE:""	 Filling with <NA>
ERROR READING VALUE:""	 Filling with <NA>
ERROR READING VALUE:""	 Filling with <NA>


Days since last Reporting Country/ Territory/Area Total confirmed  \
     reported case                                             cases   
2             <NA>                            Africa                   
3                0                      South Africa         287 796   
4                0                           Nigeria           33153   
5                0                             Ghana           24988   
6                0                           Algeria           19689   
7                1                          Cameroon           15173   
8                0                     Côƚe d͛Iǀoiƌe           12872   
9                0                             Kenya           10294   
10               0                           Senegal            8198   
11               0  Democratic Republic of the Congo            8074   
12               0                          Ethiopia            7969   
13               0                            Guinea            6170   
14               3                             Gabon            5942   
15               0                        Mauritania            5355   
16               0                        Madagascar            5080   
17               0          Central African Republic            4321   
18               0                            Malawi            2430   
19               0                              Mali            2412   
20               0                            Zambia            2283   
21               0                             Congo            2222   
22               1                       South Sudan            2148   
23               0                     Guinea-Bissau            1902   

             Total deaths Total new deaths            Transmission  
   new cases                                      classification i  
2       <NA>         <NA>             <NA>                          
3      11554         4172               93  Community transmission  
4        595          744                4  Community transmission  
5        470          139                0  Community transmission  
6        494         1018                7  Community transmission  
7          0          359                0  Community transmission  
8        106           84                0  Community transmission  
9        189          197               12  Community transmission  
10        63          150                2  Community transmission  
11        42          189                1  Community transmission  
12       203          139               11  Community transmission  
13        29           38                1  Community transmission  
14         0           46                0  Community transmission  
15        80          147                0       Clusters of cases  
16       213           37                2       Clusters of cases  
17        33           53                0  Community transmission  
18        66           39                1       Clusters of cases  
19         1          121                0  Community transmission  
20       388           82               40  Community transmission  
21       119           47                0  Community transmission  
22         0           41                0       Clusters of cases  
23        60           26                1  Community transmission




 HTML: 


